In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import acquire
import wrangle

In [2]:
df = acquire.get_telco_data()

In [3]:
df = wrangle.wrangle_telco()

In [4]:
df.head(70)

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,streaming_movies,paperless_billing,contract_type_id,payment_type_id,internet_service_type_id,churn,automatic_payment,total_charges_scaled,tenure_less_than_12,tenure_less_than_8
0,0002-ORFBO,Female,0,Yes,Yes,9,65.60,593.30,Yes,No,...,No,Yes,2,2,1,No,False,0.066294,True,False
1,0003-MKNFE,Male,0,No,No,9,59.90,542.40,Yes,Yes,...,Yes,No,1,2,1,No,False,0.060420,True,False
2,0004-TLHLJ,Male,0,No,No,4,73.90,280.85,Yes,No,...,No,Yes,1,1,2,Yes,False,0.030239,True,True
3,0011-IGKFF,Male,1,Yes,No,13,98.00,1237.85,Yes,No,...,Yes,Yes,1,1,2,Yes,False,0.140670,False,False
4,0013-EXCHZ,Female,1,Yes,No,3,83.90,267.40,Yes,No,...,No,Yes,1,2,2,Yes,False,0.028687,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0111-KLBQG,Male,1,Yes,Yes,32,93.95,2861.45,Yes,No,...,Yes,Yes,1,2,2,No,False,0.328023,False,False
66,0112-QAWRZ,Male,0,Yes,Yes,16,90.80,1442.20,Yes,Yes,...,No,Yes,1,3,2,No,True,0.164251,False,False
67,0112-QWPNC,Male,0,Yes,No,49,84.35,4059.35,Yes,Yes,...,Yes,No,2,1,1,Yes,False,0.466253,False,False
68,0114-IGABW,Female,0,Yes,No,71,58.25,4145.90,No,No phone service,...,Yes,No,3,3,1,No,True,0.476240,False,False


In [5]:
train, test = train_test_split(df, random_state=123, train_size=.80)

In [6]:
encoder = LabelEncoder()

encode_list = [
    'gender', 'partner', 'dependents', 'phone_service'
    , 'multiple_lines', 'online_security', 'online_backup'
    , 'device_protection', 'tech_support'
    , 'streaming_movies', 'streaming_tv', 'paperless_billing', 'churn'
    ]
               
               
               
for e in encode_list:
    train[e] = encoder.fit_transform(train[e])
    test[e] = encoder.transform(test[e])

# Optimize for recall

## Logit classifier without total_charges_scaled

In [7]:
# train, validate = train_test_split(train, random_state=123, test_size=.2)


# X_train = train[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_train = train[['churn']]
# X_val = validate[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_val = validate[['churn']]
# X_test = test[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_test = test[['churn']]

In [8]:
# logit = LogisticRegression(random_state = 123)
# logit.fit(X_train, y_train)

In [9]:
# y_pred = logit.predict(X_train)
# y_pred[:10]

In [10]:
# print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
#      .format(logit.score(X_train, y_train)))

In [11]:
# pd.DataFrame(confusion_matrix(y_train, y_pred))

In [12]:
# model_df = train[['churn']]
# model_df.rename(columns={'churn': 'actual'})

In [13]:
# model_df['logit_predictions'] = y_pred
# model_df.head()

In [14]:
# print(classification_report(y_train, y_pred))

## Logit classifier with tenure less than 7

In [15]:
train, validate = train_test_split(train, random_state=123, test_size=.2)


X_train = train[['contract_type_id', 'senior_citizen',  'tenure_less_than_8', 'total_charges_scaled']]
y_train = train[['churn']]
X_val = validate[['contract_type_id', 'senior_citizen', 'tenure_less_than_8', 'total_charges_scaled']]
y_val = validate[['churn']]
X_test = test[['contract_type_id', 'senior_citizen',  'tenure_less_than_8', 'total_charges_scaled']]
y_test = test[['churn']]

In [16]:
logit = LogisticRegression(random_state = 123)
logit.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
y_pred = logit.predict(X_val)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [19]:
confusion_matrix(y_val, y_pred)

array([[702, 124],
       [160, 141]])

In [20]:
train.churn.value_counts()

0    3316
1    1191
Name: churn, dtype: int64

In [21]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_val, y_val)))

Accuracy of Logistic Regression classifier on training set: 0.75


In [23]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83       826
           1       0.53      0.47      0.50       301

    accuracy                           0.75      1127
   macro avg       0.67      0.66      0.66      1127
weighted avg       0.74      0.75      0.74      1127



# Logit classifier with tenure less than 12

In [ ]:
X_train1 = train[['contract_type_id',  'total_charges_scaled', 'tenure_less_than_12']]
y_train1 = train[['churn']]
X_val1 = validate[['contract_type_id',  'total_charges_scaled', 'tenure_less_than_12']]
y_val1 = validate[['churn']]
X_test1 = test[['contract_type_id',  'total_charges_scaled', 'tenure_less_than_12']]
y_test1 = test[['churn']]

In [ ]:
logit.fit(X_train1, y_train1)

In [ ]:
y_pred1 = logit.predict(X_train1)
y_pred1[:10]

In [ ]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train1, y_train1)))

In [ ]:
print(classification_report(y_train1, y_pred1))

# Logit classifier with ss

In [ ]:
# X_train2 = train[['senior_citizen', 'contract_type_id', 'tenure_less_than_8']]
# y_train2 = train[['churn']]
# X_val2 = validate[['senior_citizen','contract_type_id',  'tenure_less_than_8']]
# y_val2 = validate[['churn']]
# X_test2 = test[['senior_citizen', 'contract_type_id', 'tenure_less_than_8']]
# y_test2 = test[['churn']]

In [ ]:
# logit.fit(X_train2, y_train2)

In [ ]:
# y_pred2 = logit.predict(X_train2)
# y_pred2[:10]

In [ ]:
# print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
#      .format(logit.score(X_train2, y_train2)))

In [ ]:
# print(classification_report(y_train1, y_pred1))

# Decision Tree with best features

In [ ]:
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=123)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_train)
y_pred[0:5]

In [ ]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

In [ ]:
confusion_matrix(y_train, y_pred)

In [ ]:
print(classification_report(y_train, y_pred))